In [22]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [23]:
sys.path.append("..")

In [24]:
from src.data import CARCADataset, load_attrs, load_ctx, load_profiles, set_datapath
from src.carca import AttrCtxEmbedding, AllEmbedding, AttrEmbedding, IdEmbedding
from src.carca import IdentityEncoding, LearnableEncoding, PositionalEncoding
from src.carca import CARCA, SelfAttentionBlock, CrossAttentionBlock, DotProduct
from src.knn import KNN
from src.train import train, evaluate
from src.utils import to, get_mask

In [25]:
DATASET = "video_games"
MODE = "sub"

In [26]:
set_datapath(f"../../data/{DATASET}")

In [27]:
attrs = load_attrs(f"dummy_all.dat")
ctx = load_ctx(f"ctx_all.dat")
user_ids, item_ids, profiles = load_profiles(f"profiles_{MODE}.txt")

In [28]:
n_items = attrs.shape[0]
n_ctx = next(iter(ctx.values())).shape[0]
n_attrs = attrs.shape[1]

In [29]:
# Hyper-parameters
seq_len = 50
batch_size = 256

In [30]:
test_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="test"
)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

In [31]:
device = "cuda"
model = torch.load("../results/video_games/sub/x_identity_id_dot/005_0.6076_0.3706.pth")
model = model.eval().to(device)

In [32]:
k = 10
results = []

with torch.no_grad():
    for batch in test_loader:
        p_x, p_a, p_c, o_x, o_a, o_c, y_true = to(*batch, device=device)
        y_pred = model.forward(profile=(p_x, p_a, p_c), targets=[(o_x, o_a, o_c)])
        
        y_pred_sort, idxs = torch.sort(y_pred, descending=True)
        y_true_sort = torch.gather(y_true, dim=1, index=idxs)
        
        hrs = torch.sum(y_true_sort[:, :k], dim=-1).tolist()
        ranks = torch.nonzero(y_true_sort)[:, 1]
        ndcgs = torch.where(ranks < 10, 1.0 / torch.log2(ranks + 2), 0.0).tolist()
        nonzeros = torch.count_nonzero(p_x, dim=-1).tolist()

        results.extend(list(zip(hrs, ndcgs, nonzeros)))

In [103]:
evaluate(model, test_loader, device, 10)

(0.7749788529859584, 0.5705197324770555, 0.3198920911524719)

In [33]:
with open("../results/video_games/sub/sbert_identity_all_dot/results_per_seqlen.txt", "w") as f:
    f.writelines([f"{x[0]} {x[1]} {x[2]}\n" for x in results])